In [ ]:
#ignore the stuff here, we arnet using it

In [1]:
pip install python-dotenv



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve the API key
api_key = os.getenv('API_KEY')

In [25]:
import pandas as pd


In [13]:
api_key

'24DD4E29-B6FF-31DC-9066-43A4107B0C42'

In [15]:
Commodities = ['ALMONDS', 'APPLES', 'AVOCADOS', 'BEANS', 'BLACKBERRIES', 'BROCCOLI', 'CHERRIES', 'CORN', 'GRAPES', 'LEMONS', 'LETTUCE', 'OLIVES', 'ORANGES', 'PEACHES', 'PEARS', 'PISTACHIOS', 'PLUMS', 'POTATOES', 'PLUMS', 'POTATOES', 'PRUNES', 'RASPBERRIES', 'RICE', 'SORGHUM', 'SPINACH', 'STRAWBERRIES', 'SUNFLOWER', 'TOMATOES', 'WALNUTS', 'WHEAT']

In [ ]:
# Product	Labor Cost	Total Cost
# 0	alfalas	837.00	5961.00 check
# 1	almonds	2104.00	12308.00
# 2	apples	4440.00	7060.00
# 3	avocados	1863.00	10018.00
# 4	beans	569.22	2886.88
# 5	blackberries	28945.00	62179.00
# 6	broccoli	3942.00	14200.00
# 7	cherries	1273.00	2967.00
# 8	corn	409.00	2285.00
# 9	grape-raisins	2480.00	3646.00
# 10	grapes-table	24426.00	31441.00
# 11	grapes-wine	9009.00	20350.00
# 12	lemons	1047.00	5235.00
# 13	lettuce	2514.00	18274.00
# 14	olive-oil	173.00	561.00
# 15	olives	595.00	4668.00
# 16	oranges	482.00	2800.00
# 17	orchardgrass-hay	83.00	589.00
# 18	pastures	0.00	1469.00
# 19	peaches	3381.00	7501.00
# 20	pears	584.00	5652.00
# 21	pistachios	356.00	2747.00
# 22	plums	1275.00	2570.00
# 23	potatoes	608.20	3275.96
# 24	prunes	624.00	5110.00
# 25	raspberries	16685.00	26881.00
# 26	rice	139.00	1388.00
# 27	sorghum	108.00	1056.00
# 28	spinach	722.00	4336.00
# 29	strawberries	18843.00	49900.00
# 30	sunflowers	176.00	771.00
# 31	tomatoes	812.84	6897.00
# 32	walnuts	578.00	5702.00
# 33	wheat	

In [139]:
import requests
import pandas as pd
base_url = 'https://quickstats.nass.usda.gov/api/api_GET/'

def fetch_data(commodity):
    # Set up the parameters for the API request
    params = {
        'key': api_key,
        'source_desc': 'CENSUS',
        'sector_desc': 'CROPS',
        'commodity_desc':commodity,
        'statisticcat_desc': 'AREA HARVESTED',
        'year': 2016,
        'agg_level_desc': 'STATE',
        'freq_desc': 'Annual',
        'format': 'JSON'  # Desired response format
    }

    # Send the GET request
    response = requests.get(base_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Parse the JSON response
        return data
    else:
        print(f"Error fetching data {response.status_code}")
        print("HUH")
        return None
    
#stdrvmkcr


In [144]:
corn = fetch_data('ALMONDS')
total_df = pd.DataFrame(corn['data'])

In [145]:
for i in range(1, len(Commodities)):
    current = fetch_data(Commodities[i])
    current_df = pd.DataFrame(current['data'])
    total_df = pd.concat([total_df, current_df], axis=0, ignore_index=True)


In [146]:
filter_df = total_df[total_df['unit_desc'] == 'ACRES']

In [147]:
filter_df.columns

Index(['state_ansi', 'week_ending', 'begin_code', 'county_code', 'county_name',
       'end_code', 'Value', 'asd_code', 'country_name', 'state_fips_code',
       'location_desc', 'reference_period_desc', 'state_alpha', 'county_ansi',
       'domain_desc', 'util_practice_desc', 'congr_district_code', 'freq_desc',
       'unit_desc', 'year', 'region_desc', 'group_desc', 'commodity_desc',
       'CV (%)', 'domaincat_desc', 'short_desc', 'statisticcat_desc',
       'source_desc', 'zip_5', 'class_desc', 'watershed_code',
       'agg_level_desc', 'watershed_desc', 'country_code', 'state_name',
       'load_time', 'asd_desc', 'prodn_practice_desc', 'sector_desc'],
      dtype='object')

In [148]:
filter_df.to_csv('api_2021.csv')